In [1]:
import sys, os, time
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib tk
import matplotlib.pyplot as plt

import tnt_util as util
from tnt_util import adict, idict, xset, collate, load, render_dict, get_object, save, Logger, DigitalLog
from tnt_setup import init_gamestate, setup_phase
from tnt_cards import load_card_decks, draw_cards
from collections import namedtuple
import random
from itertools import chain, product

from flask_app import *

In [2]:
ping()

'Backend active: use "init" to init game'

In [3]:
out = init_game(debug=True)

Beginning phase: Setup
adict(actions:xset({('Britain', xset({(xset({'London', 'Bombay', 'Glasgow', 'Delhi', 'Suez'}), xset({'AirForce', 'Carrier', 'Submarine', 'Infantry', 'Fortress', 'Tank', 'Fleet'}))})), ('France', xset({(xset({'Marseille', 'Paris', 'Algiers'}), xset({'AirForce', 'Carrier', 'Submarine', 'Infantry', 'Fortress', 'Tank', 'Fleet'}))}))}), messages:, created:<class 'structures.arg_dict.adict'>(Ottawa, New_York, Washington, Irish_Sea, North_Atlantic_Ocean, Mid_Atlantic_Ocean, South_Atlantic_Ocean, Rio_de_Janeiro, Dakar, Morocco, Algiers, Tunisia, Sfax, Gibraltar, Madeira_Sea, Lisbon, Madrid, Leon, Barcelona, Azores, Bay_of_Biscay, Dublin, Irminger_Sea, Iceland, Greenland_Sea, Norwegian_Sea, Nordkapp_Sea, North_Sea, English_Channel, Glasgow, London, Paris, Lorraine, Gascony, Marseille, Western_Mediterranean, Sardinia, Tyrrhenian_Sea, Malta, Sicily, Central_Mediterranean, Adriatic_Sea, Milan, Rome, Venice, Taranto, Tripoli, Cyrenaica, Amsterdam, Vienna, Munich, Ruhr, Berlin

In [4]:
player = 'West'

In [5]:
out = get_status(player)

In [8]:
render_dict(out)

In [10]:
# action = random.choice(list(util.decode_actions(out.actions)))
# action

In [11]:
#out = take_action(player, action)

In [19]:
out = take_action(player, ('Britain', 'Paris', 'Infantry'))

In [14]:
for _ in range(10):
    if 'error' in out:
        print(out.error)
        break
    elif 'actions' not in out:
        #render_dict(out)
        print('Player {} is waiting'.format(player))
        break
    action = random.choice(list(util.decode_actions(out.actions)))
    print(action)
    out = take_action(player, action)

In [20]:
G = get_G()
render_dict(G)

In [33]:
W = get_waiting_actions()
render_dict(W)